In [14]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
df = pd.read_csv("usa_000132.csv")
df = df.dropna(subset=["INCTOT","FTOTINC"])

In [15]:
# Filter rows for key column data
df = df[df["METRO"] != 0] # remove non-metro areas
df = df[(df["AGE"] >= 18) & (df["AGE"] <= 64)]  # keep age 18–64
df = df[df["EDUC"].isin([10, 11])]  # keep individuals with a bachelor's degree or higher
df = df[df["CLASSWKR"] != 0]  # remove n/a class of worker

In [16]:
# Dummy Variables
df["IND_N"] = np.where(df["IND"] == 7390, 1, 2) # technical consulting industry = 1, else = 2
df["MARITAL_STATUS"] = np.where(df["MARST"].isin([1, 2]), 1, 2) # married, spouse present or absent = 1, not married = 2
df["RACE_N"] = np.where(df["RACE"] == 1, 1, 2) # white = 1, else = 2
df["ENGLISH_SPEAKING_STATUS"] = np.where(df["SPEAKENG"].isin([2, 3, 4, 5, 6]), 1, 2) # speaks English well to barely speaks = 1, doesn't speak English = 2

In [17]:
# Occupation categories
def recode_occ(x):
    if 0000 <= x <= 960:
        return 1
    elif 961 <= x <= 1980:
        return 2
    elif 1981 <= x <= 2920:
        return 3
    elif 2921 <= x <= 3550:
        return 4
    elif 3551 <= x <= 4655:
        return 5
    elif 4656 <= x <= 5940:
        return 6
    elif 5941 <= x <= 7640:
        return 7
    elif 7641 <= x <= 8990:
        return 8
    elif 8991 <= x <= 9920:
        return 9
    else:
        return np.nan  # in case occ=0 or out of range
df["OCCUPATION_GROUP"] = df["OCC"].apply(recode_occ)
df = df.dropna(subset=["OCCUPATION_GROUP"])

In [18]:
# Rename Income Variables & Log Tranformations to adjust for skewness
df = df.rename(columns={
    "INCTOT": "PERSONAL_INCOME",
    "FTOTINC": "FAMILY_INCOME"
})     # Rename income variables

df = df[df["PERSONAL_INCOME"] >= 0]  # remove negative personal income values
df = df[df["FAMILY_INCOME"] >= 0]  # remove negative family income values

df["LOG_OF_PERSONAL_INCOME"] = np.log(df["PERSONAL_INCOME"].replace({0: np.nan}))
df["LOG_OF_FAMILY_INCOME"] = np.log(df["FAMILY_INCOME"].replace({0: np.nan}))
df = df.dropna(subset=["LOG_OF_PERSONAL_INCOME","LOG_OF_FAMILY_INCOME"])

In [19]:
df = df.rename(columns={
    "IND_N": "TECHNICAL_CONSULTING_DUMMY",
    "METRO": "METROPOLITAN_STATUS",
    "CITIZEN": "CITIZENSHIP_STATUS",
    "EDUC": "EDUCATION_LEVEL",
    "CLASSWKR": "EMPLOYMENT_STATUS",
    "WKSWORK1": "WEEKS_WORKED_LAST_YEAR",
    "VETSTAT": "VETERAN_STATUS"
})  # Rename other variables

In [20]:
print(df.shape)
print(df.head())

(87706, 23)
     YEAR  METROPOLITAN_STATUS  SEX  AGE  MARST  RACE  CITIZENSHIP_STATUS  \
74   2022                    4    1   54      3     3                   0   
125  2022                    4    1   54      3     3                   0   
338  2022                    4    1   42      2     1                   0   
648  2022                    4    1   42      2     1                   0   
839  2022                    4    2   33      6     2                   0   

     SPEAKENG  EDUCATION_LEVEL  EMPLOYMENT_STATUS   OCC   IND  \
74          4               10                  2  4000  8680   
125         4               10                  2  4000  8680   
338         3               11                  2  4220  1180   
648         3               11                  2  4220  1180   
839         3               10                  2  4020  5090   

     WEEKS_WORKED_LAST_YEAR  PERSONAL_INCOME  FAMILY_INCOME  VETERAN_STATUS  \
74                       52            26000        999

In [21]:
df.to_csv("usa_clean.csv", index=False)

In [22]:
vars_1 = ["PERSONAL_INCOME", "FAMILY_INCOME", "LOG_OF_PERSONAL_INCOME", "LOG_OF_FAMILY_INCOME", "TECHNICAL_CONSULTING_DUMMY", "OCCUPATION_GROUP", "SEX", "AGE", "MARITAL_STATUS", "RACE_N", "METROPOLITAN_STATUS", "CITIZENSHIP_STATUS", "ENGLISH_SPEAKING_STATUS", "EDUCATION_LEVEL", "EMPLOYMENT_STATUS", "VETERAN_STATUS", "WEEKS_WORKED_LAST_YEAR"]

def descriptive_stats(series):
    return pd.Series({
        "count": series.count(),
        "mean": series.mean(),
        "min": series.min(),
        "max": series.max(),
        "std_dev": series.std()
    })

summary_table = df[vars_1].apply(descriptive_stats)
summary_table = summary_table.T
print(summary_table)

                              count           mean   min           max  \
PERSONAL_INCOME             87706.0  109794.552174   1.0  1.557000e+06   
FAMILY_INCOME               87706.0  300013.147231   1.0  9.999999e+06   
LOG_OF_PERSONAL_INCOME      87706.0      11.113559   0.0  1.425827e+01   
LOG_OF_FAMILY_INCOME        87706.0      11.893055   0.0  1.611810e+01   
TECHNICAL_CONSULTING_DUMMY  87706.0       1.977470   1.0  2.000000e+00   
OCCUPATION_GROUP            87706.0       3.136137   1.0  9.000000e+00   
SEX                         87706.0       1.516236   1.0  2.000000e+00   
AGE                         87706.0      42.364958  19.0  6.400000e+01   
MARITAL_STATUS              87706.0       1.397521   1.0  2.000000e+00   
RACE_N                      87706.0       1.454439   1.0  2.000000e+00   
METROPOLITAN_STATUS         87706.0       3.213885   1.0  4.000000e+00   
CITIZENSHIP_STATUS          87706.0       0.617837   0.0  3.000000e+00   
ENGLISH_SPEAKING_STATUS     87706.0   

In [23]:
summary_table.to_csv("usa_summary.csv")

In [24]:
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
model = smf.ols("LOG_OF_PERSONAL_INCOME ~ C(TECHNICAL_CONSULTING_DUMMY) + C(METROPOLITAN_STATUS)", data=df).fit()
# create a mapping of old coefficient names to friendly names
coef_rename = {
    "C(TECHNICAL_CONSULTING_DUMMY)[T.2]": "W(Control) Tech Consulting Industry",
    "C(METROPOLITAN_STATUS)[T.2]": "Large City x Tech Consulting",
    "C(METROPOLITAN_STATUS)[T.3]": "Small City x Tech Consulting",
    "C(METROPOLITAN_STATUS)[T.4]": "Unknown City x Tech Consulting"
}

# create a nicer summary table
summary_df = model.summary2().tables[1]  # coefficient table
summary_df = summary_df.rename(index=coef_rename)
print(summary_df)
# --- Save regression table ---
results_df = pd.DataFrame({
    "Coefficient": model.params,
    "Std_Error": model.bse,
    "t_value": model.tvalues,
    "p_value": model.pvalues
})
results_df = results_df.rename(index=coef_rename)
results_df.to_csv("regression_results.csv", index=True)

                                         Coef.  Std.Err.           t  \
Intercept                            10.961311  0.033137  330.789556   
W(Control) Tech Consulting Industry  -0.180229  0.025955   -6.943949   
Large City x Tech Consulting          0.379566  0.022439   16.915511   
Small City x Tech Consulting          0.233369  0.022636   10.309423   
Unknown City x Tech Consulting        0.364298  0.021675   16.807085   

                                            P>|t|     [0.025     0.975]  
Intercept                            0.000000e+00  10.896363  11.026259  
W(Control) Tech Consulting Industry  3.839284e-12  -0.231100  -0.129358  
Large City x Tech Consulting         4.371970e-64   0.335586   0.423546  
Small City x Tech Consulting         6.601231e-25   0.189002   0.277736  
Unknown City x Tech Consulting       2.721798e-63   0.321815   0.406782  
